In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle
import random
import cv2
#from tqmd import tqmd
DIRECTORY = "img"
FRUITS = ["bananas"]
CATEGORIES = ["good", "bad"]
IMG_SIZE = 70

## Falta cargar/crear una base de datos. sólo copié lo del profe

# Cargar las imágenes

In [ ]:
images = []
for fruit in FRUITS:
    for cat in CATEGORIES:
        path = os.path.join(DIRECTORY,fruit,cat)
        class_num = CATEGORIES.index(cat) #good = 0  | bad = 1
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path,img) ,cv2.IMREAD_COLOR)
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
                images.append([new_array, class_num])
            except Exception as e:
                pass

# Imágenes a matrices

In [ ]:
X = []
y = []
random.shuffle(images)
for i, clase in images:
    X.append(i)
    y.append(clase)
    
salida = open("X.pickel","wb")
pickle.dump(X, salida)
salida.close()
salida = open("y.pickel","wb")
pickle.dump(y, salida)
salida.close()

# Modelo

In [ ]:
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import Adam

In [ ]:
X = np.array(X)
X = X/X.max() ##NO CORRER MÁS DE UNA VEZ
X = X.reshape(-1,70,70,1)

In [ ]:
model = Sequential()
model.add()#...
model.add()#...
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [ ]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy',metrics=['accuracy'])
model.fit(X,y,batch_size=32, epochs=10, validation_split=.25) #HAY QUE CAMBIAR LAS EPOCAS